<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_de_Tensorflow-2.0_Guia_completa_para_el_Nuevo_Tensorflow/blob/master/Capitulo_6_transfer_learning_2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet                                                                                                                    # IGNORE_COPYRIGHT: cleared by OSS licensing
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Transferencia de aprendizaje y puesta a punto

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/transfer_learning"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/transfer_learning.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

En este tutorial, aprenderá a clasificar imágenes de gatos y perros mediante el aprendizaje de transferencia de una red pre-entrenada.

Un modelo pre-entrenado es una red guardada que fue entrenada previamente en un gran conjunto de datos, típicamente en una tarea de clasificación de imágenes a gran escala. Puede utilizar el modelo previamente entrenado tal cual o utilizar el aprendizaje de transferencia para personalizar este modelo para una tarea determinada.

La intuición detrás del aprendizaje de transferencia para la clasificación de imágenes es que si un modelo está entrenado en un conjunto de datos lo suficientemente grande y general, este modelo servirá efectivamente como un modelo genérico del mundo visual. Luego puede aprovechar estos mapas de funciones aprendidas sin tener que comenzar desde cero entrenando un modelo grande en un conjunto de datos grande.

En este cuaderno, intentará dos formas de personalizar un modelo previamente entrenado:

1. Extracción de características: utilice las representaciones aprendidas por una red anterior para extraer características significativas de nuevas muestras. Simplemente agregue un nuevo clasificador, que se entrenará desde cero, encima del modelo previamente entrenado para que pueda reutilizar los mapas de características aprendidos previamente para el conjunto de datos.

No es necesario (re) entrenar todo el modelo. La red convolucional base ya contiene características que son genéricamente útiles para clasificar imágenes. Sin embargo, la parte final de la clasificación del modelo preentrenado es específica de la tarea de clasificación original y, posteriormente, específica del conjunto de clases en las que se formó el modelo.

1. Ajuste fino: Descongele algunas de las capas superiores de una base de modelo congelada y entrene conjuntamente las capas de clasificador recién agregadas y las últimas capas del modelo base. Esto nos permite "ajustar" las representaciones de características de orden superior en el modelo base para hacerlas más relevantes para la tarea específica.

Seguirás el flujo de trabajo general de aprendizaje automático

1. Examinar y comprender los datos.
1. Construya una tubería de entrada, en este caso usando Keras ImageDataGenerator
1. Componer el modelo
   * Carga en el modelo base previamente entrenado (y pesos pretratados)
   * Apila las capas de clasificación en la parte superior
1. Entrenar a la modelo
1. Evaluar modelo


In [ ]:
!pip install tf-nightly

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

## Data preprocessing

### Data download

Descarga de datos
En este tutorial, utilizará un conjunto de datos que contiene varios miles de imágenes de gatos y perros. Descargue y extraiga un archivo zip que contenga las imágenes, luego cree un `tf.data.Dataset` para capacitación y validación utilizando la utilidad `tf.keras.preprocessing.image_dataset_from_directory` Puede obtener más información sobre cómo cargar imágenes en este tutorial [tutorial](https://www.tensorflow.org/tutorials/load_data/images).

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

# carpetas del comprimindo
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

In [ ]:
validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

Muestre las primeras nueve imágenes y etiquetas del conjunto de entrenami

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(20, 20))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Como el conjunto de datos original no contiene un conjunto de prueba, creará uno. Para hacerlo, determine cuántos lotes de datos están disponibles en el conjunto de validación usando ```tf.data.experimental.cardinality```, luego mueva el 20% de ellos a un conjunto de prueba.

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
print("val_batches:",val_batches)
test_dataset = validation_dataset.take(val_batches // 5)
print("test_dataset:",test_dataset)
validation_dataset = validation_dataset.skip(val_batches // 5)
print("validation_dataset:",validation_dataset)

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

### Configurar el conjunto de datos para el rendimiento

Utilice la captación previa almacenada en búfer para cargar imágenes desde el disco sin que la E / S se bloquee. Para obtener más información sobre este método, consulte la guía de  . [rendimiento de datos](https://www.tensorflow.org/guide/data_performance) 

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Usar aumento de datos

Cuando no tiene un conjunto de datos de imágenes grandes, es una buena práctica introducir artificialmente la diversidad de muestras aplicando transformaciones aleatorias pero realistas a las imágenes de entrenamiento, como la rotación y el giro horizontal. Esto ayuda a exponer el modelo a diferentes aspectos de los datos de entrenamiento y a reducir el  [sobreajuste](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit). Puede obtener más información sobre el aumento de datos en este tutorial [tutorial](https://www.tensorflow.org/tutorials/images/data_augmentation).

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

Nota: Estas capas están activas solo durante el entrenamiento, cuando llamas a `model.fit`. Están inactivos cuando el modelo se usa en modo de inferencia en `model.evaulate` o `model.fit`.

Apliquemos repetidamente estas capas a la misma imagen y veamos el resultado.

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### Reescalar valores de píxeles

En un momento, descargará `tf.keras.applications.MobileNetV2` para usarlo como su modelo base. Este modelo espera valores de píxeles en `[-1,1]`, pero en este punto, los valores de píxeles en sus imágenes están en `[0-255]`. Para reescalarlos, use el método de preprocesamiento incluido con el modelo.

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

Nota: Como alternativa, puede cambiar la escala de los valores de píxeles de `[0,255]` a `[-1, 1]` utilizando una capa de reescalado [Rescaling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling) .

 

In [ ]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

Nota: Si usa otras aplicaciones `tf.keras.applications` , asegúrese de verificar el documento API para determinar si esperan píxeles en `[-1,1]` o `[0,1]` , o use la función `preprocess_input ` incluida.

## Cree el modelo base a partir de las redes de comunicación previamente capacitadas.
**Creará** el modelo base a partir del modelo **MobileNet V2** desarrollado en Google. Esto está pre-entrenado en el conjunto de datos ImageNet, un gran conjunto de datos que consta de imágenes 1.4M y 1000 clases. ImageNet es un conjunto de datos de capacitación en investigación con una amplia variedad de categorías como `jackfruit` y `syringe` . Esta base de conocimiento nos ayudará a clasificar gatos y perros de nuestro conjunto de datos específico.

Primero, debe elegir qué capa de MobileNet V2 usará para la extracción de características. La última capa de clasificación (en "arriba", ya que la mayoría de los diagramas de modelos de aprendizaje automático van de abajo hacia arriba) no es muy útil. En su lugar, seguirá la práctica común de depender de la última capa antes de la operación de aplanar. Esta capa se llama "capa de cuello de botella". Las características de la capa de cuello de botella conservan más generalidad en comparación con la capa final / superior.

Primero, cree una instancia de un modelo MobileNet V2 precargado con pesas entrenadas en ImageNet. Al especificar el argumento **include_top = False** , carga una red que no incluye las capas de clasificación en la parte superior, lo que es ideal para la extracción de características.

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Este extractor de funciones convierte cada imagen de **160x160x3** en un bloque de características de **5x5x1280** . Veamos qué le hace a un lote de imágenes de ejemplo:

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Extracción de características
En este paso, congelará la base convolucional creada a partir del paso anterior y la usará como un extractor de características. Además, agrega un clasificador encima y entrena al clasificador de nivel superior.

### Congelar la base convolucional

Es importante congelar la base convolucional antes de compilar y entrenar el modelo. La congelación (al establecer layer.trainable = False) evita que los pesos en una capa dada se actualicen durante el entrenamiento. MobileNet V2 tiene muchas capas, por lo que establecer el indicador trainable todo el modelo en False las congelará todas.

In [ ]:
base_model.trainable = False

### Nota importante sobre las capas de BatchNormalization
Muchos modelos contienen capas`tf.keras.layers.BatchNormalization` Esta capa es un caso especial y se deben tomar precauciones en el contexto del ajuste, como se muestra más adelante en este tutorial.

Cuando establece `layer.trainable = False` , la capa `BatchNormalization` se ejecutará en modo de inferencia y no actualizará sus estadísticas de media y varianza.

Cuando descongela un modelo que contiene capas de BatchNormalization para realizar un ajuste fino, debe mantener las capas de BatchNormalization en modo de inferencia pasando `training = False` al llamar al modelo base. De lo contrario, las actualizaciones aplicadas a los pesos no entrenables destruirán lo que el modelo ha aprendido.

Para más detalles, consulte la guía de aprendizaje Transferir . [Transfer learning guide](https://www.tensorflow.org/guide/keras/transfer_learning).

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

### Agregar una cabeza de clasificación
Para generar predicciones a partir del bloque de características, `tf.keras.layers.GlobalAveragePooling2D` las ubicaciones espaciales especiales `5x5` , usando una capa `tf.keras.layers.GlobalAveragePooling2D` para convertir las características en un solo vector de 1280 elementos por imagen.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

Aplique una capa `tf.keras.layers.Dense` para convertir estas características en una sola predicción por imagen. No necesita una función de activación aquí porque esta predicción se tratará como un `logit` o un valor de predicción sin procesar. Los números positivos predicen la clase 1, los números negativos predicen la clase 0.

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

Cree un modelo encadenando las capas de aumento de datos, reescalamiento, base_model y extractor de características utilizando la API funcional de Keras [Keras Functional API](https://www.tensorflow.org/guide/keras/functional) . Como se mencionó anteriormente, use training = False ya que nuestro modelo contiene una capa BatchNormalization. . As previously mentioned, use training=False as our model contains a BatchNormalization layer.

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

### Compila el modelo
Compile el modelo antes de entrenarlo. Como hay dos clases, use una pérdida binaria de entropía cruzada con `from_logits=True` ya que el modelo proporciona una salida lineal.

In [ ]:
base_learning_rate = 0.0001 # RADIO DE APRENDIZAJE MUY BAJO
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Los parámetros de 2.5M en MobileNet están congelados, pero hay 1.2K parámetros entrenables en la capa densa. Estos se dividen en dos `tf.Variable` Objetos `tf.Variable` , los pesos y sesgos.

In [ ]:
len(model.trainable_variables)

### Entrenar a la modelo 
Después de entrenar durante 10 épocas, debería ver una precisión de ~ 94% en el conjunto de validación.


In [ ]:
initial_epochs = 10

# EVALUACION INICIAL DEL MODELO
loss0, accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

### Curvas de aprendizaje
Echemos un vistazo a las curvas de aprendizaje del entrenamiento y la precisión / pérdida de validación cuando se usa el modelo base MobileNet V2 como un extractor de características fijas.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Nota: Si se pregunta por qué las métricas de validación son claramente mejores que las métricas de entrenamiento, el factor principal es porque las capas como `tf.keras.layers.BatchNormalization` y `tf.keras.layers.Dropout` afectan la precisión durante el entrenamiento. Se desactivan al calcular la pérdida de validación.


En menor medida, también se debe a que las métricas de capacitación informan el promedio de una época, mientras que las métricas de validación se evalúan después de la época, por lo que las métricas de validación ven un modelo que se ha entrenado un poco más.


## Sintonia FINA
En el experimento de extracción de características, solo estaba entrenando unas pocas capas sobre un modelo base MobileNet V2. Los pesos de la red pre-entrenada **no** se actualizaron durante el entrenamiento.

Una forma de aumentar aún más el rendimiento es entrenar (o "ajustar") los pesos de las capas superiores del modelo pre-entrenado junto con el entrenamiento del clasificador que agregó. El proceso de capacitación obligará a ajustar los pesos de los mapas de características genéricas a las características asociadas específicamente con el conjunto de datos.

*Nota: Esto solo debe intentarse después de haber entrenado el clasificador de nivel superior con el modelo pre-entrenado establecido en no entrenable. Si agrega un clasificador inicializado aleatoriamente encima de un modelo pre-entrenado e intenta entrenar todas las capas conjuntamente, la magnitud de las actualizaciones de gradiente será demasiado grande (debido a los pesos aleatorios del clasificador) y su modelo pre-entrenado será olvida lo que ha aprendido.*

Además, debe intentar ajustar una pequeña cantidad de capas superiores en lugar de todo el modelo MobileNet. En la mayoría de las redes convolucionales, cuanto más arriba está una capa, más especializada es. Las primeras capas aprenden características muy simples y genéricas que se generalizan a casi todos los tipos de imágenes. A medida que avanza, las características son cada vez más específicas para el conjunto de datos en el que se formó el modelo. El objetivo del ajuste fino es adaptar estas características especializadas para trabajar con el nuevo conjunto de datos, en lugar de sobrescribir el aprendizaje genérico.

### Un-freeze the top layers of the model


### Descongele las capas superiores del modelo
Todo lo que necesita hacer es descongelar el `base_model` y configurar las capas inferiores para que no se puedan entrenar. Luego, debe volver a compilar el modelo (necesario para que estos cambios surtan efecto) y reanudar la capacitación.

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

### Compila el modelo

Como está entrenando un modelo mucho más grande y desea readaptar los pesos entrenados, es importante utilizar una tasa de aprendizaje más baja en esta etapa. De lo contrario, su modelo podría sobreajustar muy rápidamente.

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Continuar entrenando al modelo

Si entrenó antes para la convergencia, este paso mejorará su precisión en unos pocos puntos porcentuales.

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Echemos un vistazo a las curvas de aprendizaje del entrenamiento y la precisión / pérdida de validación cuando ajustemos las últimas capas del modelo base MobileNet V2 y entrenemos el clasificador sobre él. La pérdida de validación es mucho más alta que la pérdida de entrenamiento, por lo que puede obtener algo de sobreajuste.

También puede obtener algo de sobreajuste, ya que el nuevo conjunto de entrenamiento es relativamente pequeño y similar a los conjuntos de datos originales de MobileNet V2.

Después de un ajuste fino, el modelo casi alcanza el 98% de precisión en el conjunto de validación.


In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Evaluación y predicción. 

Finalmente, puede verificar el rendimiento del modelo en los nuevos datos utilizando el conjunto de prueba.

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)
print('Test loss :', loss)

## Y ahora está todo listo para usar este modelo para predecir si su mascota es un gato o un perro.

In [ ]:
# Recuperar un lote de imágenes del conjunto de prueba
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Aplicar un sigmoide ya que nuestro modelo devuelve logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(20):
  ax = plt.subplot(5, 4, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

## Resumen
Uso de un modelo previamente entrenado para la extracción de características : cuando se trabaja con un conjunto de datos pequeño, es una práctica común aprovechar las características aprendidas por un modelo entrenado en un conjunto de datos más grande en el mismo dominio. Esto se hace creando instancias del modelo pre-entrenado y agregando un clasificador completamente conectado en la parte superior. El modelo pre-entrenado está "congelado" y solo los pesos del clasificador se actualizan durante el entrenamiento. En este caso, la base convolucional extrajo todas las características asociadas con cada imagen y solo entrenó a un clasificador que determina la clase de imagen dado ese conjunto de características extraídas.

Ajuste fino de un modelo pre-entrenado : para mejorar aún más el rendimiento, uno puede querer reutilizar las capas de nivel superior de los modelos pre-entrenados para el nuevo conjunto de datos a través del ajuste fino. En este caso, ajustó sus pesos de modo que su modelo aprendió características de alto nivel específicas del conjunto de datos. Esta técnica generalmente se recomienda cuando el conjunto de datos de entrenamiento es grande y muy similar al conjunto de datos original en el que se entrenó el modelo pre-entrenado.

Para obtener más información, visite la guía de aprendizaje Transferir .
To learn more, visit the [Transfer learning guide](https://www.tensorflow.org/guide/keras/transfer_learning).
